In [1]:
import math
import random
import string
import sys
import time
import numpy as np
import torch

import matplotlib.pyplot as plt
import src
from envs import build_env
#from src.evaluator import Evaluator
from src.slurm import init_signal_handler, init_distributed_mode
from t2.realtime_trainer import RealtimeTrainer
from src.utils import initialize_exp
from t2.utils import get_parser

from t2.transformer import build_transformer
argv = [
        '--exp_name', 'first_train', 
        '--tasks', 'add_dataset', 
        '--n_enc_layers', '16', 
        '--n_heads', '4', 
        '--sinusoidal_embeddings', 'true', 
        '--num_workers', '4', 
        '--eval_onl', '0', 
        '--save_periodic', '0', 
        '--epoch_size', '10000', 
        '--batch_size', '32', 
        '--dropout', '0.1', 
        '--attention_dropout', '0.1', 
        '--emb_dim', '64', 
        '--bottleneck_dim', '64', 
        '--nn_output', '1', 
        '--input_seq_length', '47', 
        '--share_inout_emb', 'false'
]

parser = get_parser()

params = parser.parse_args(argv)
print(params)

bs = params.batch_size
#dim = params.emb_dim

init_distributed_mode(params)
logger = initialize_exp(params)

# CPU / CUDA
if params.cpu:
    assert not params.multi_gpu
else:
    assert torch.cuda.is_available()
    
src.utils.CUDA = not params.cpu

env = build_env(params)

# modules = build_transformer(env, params, 'cuda')
# trainer = RealtimeTrainer(modules, env, params)

trainers=[]
number_of_models=2
for i in range(number_of_models):
    params.my_device='cuda:' + str(i)
    trainer = RealtimeTrainer(build_transformer(env, params), env, params)
    trainers += [trainer]



SAIEnvironment created
Namespace(dump_path='./dumped/', exp_name='first_train', save_periodic=0, exp_id='dmytro_job', fp16=False, amp=-1, emb_dim=64, bottleneck_dim=64, nn_output=1, input_seq_length=47, n_enc_layers=16, n_dec_layers=4, n_heads=4, dropout=0.1, attention_dropout=0.1, share_inout_emb=False, sinusoidal_embeddings=True, env_base_seed=0, max_len=512, batch_size=32, optimizer='adam,lr=0.0001', clip_grad_norm=5, epoch_size=10000, max_epoch=100000, stopping_criterion='', validation_metrics='', accumulate_gradients=1, num_workers=4, same_nb_ops_per_batch=False, reload_size=-1, env_name='sai', tasks='add_dataset', beam_eval=False, beam_size=1, beam_length_penalty=1, beam_early_stopping=True, reload_model='', reload_checkpoint='', eval_only=False, eval_verbose=0, eval_verbose_print=False, debug_slurm=False, debug=False, cpu=False, local_rank=-1, master_port=-1)
SLURM job: False
0 - Number of nodes: 1
0 - Node ID        : 0
0 - Local rank     : 0
0 - Global rank    : 0
0 - World si

INFO - 01/06/23 12:35:59 - 0:00:00 - ============ Initialized logger ============
INFO - 01/06/23 12:35:59 - 0:00:00 - accumulate_gradients: 1
                                     amp: -1
                                     attention_dropout: 0.1
                                     batch_size: 32
                                     beam_early_stopping: True
                                     beam_eval: False
                                     beam_length_penalty: 1
                                     beam_size: 1
                                     bottleneck_dim: 64
                                     clip_grad_norm: 5
                                     command: python F:\Programs\anaconda3\envs\ai\lib\site-packages\ipykernel_launcher.py '-f' 'C:\Users\Dmytro\AppData\Roaming\jupyter\runtime\kernel-6544956e-49e2-49a1-841d-3d53f39d231b.json' --exp_id "dmytro_job"
                                     cpu: False
                                     debug: False
               

In [2]:
#!nvidia-smi

In [3]:
def act(inp, trainer):
    inp = join_sai(inp)
    lst = []
    for _ in range(bs):
        x = inp
        lst += [x]
        
    return trainer.act(lst)[0]

def to_sai_str(p):
    return join_sai(p.data)


def join_sai(data):
    return ' '.join(data)

def str_diff(s1, s2):
    diff = abs(len(s1) - len(s2))
    for i in range(min(len(s1), len(s2))):
        diff += 1 if s1[i] != s2[i] else 0
    return diff


In [4]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
a = join_sai(A)

In [5]:
start_time = time.time()

losses = []
iterations_number = 10

for it in range(iterations_number):
    for trainer in trainers:
            for _ in range(params.batch_size):
                loss = trainer.learn(a, a)

end_time = time.time()
elapsed_time = end_time - start_time
av_time = elapsed_time/iterations_number/len(trainers)

print(f'Av time per iteration per trainer: {elapsed_time/iterations_number:.2f} seconds')


F:\workspace\ai\Transformers1\src\optim.py:72: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\utils\python_arg_parser.cpp:1418.)
  exp_avg.mul_(beta1).add_(1 - beta1, grad)
INFO - 01/06/23 12:36:06 - 0:00:07 - learning: av-score=0.06981382978723405, device=cuda:0
INFO - 01/06/23 12:36:07 - 0:00:08 - learning: av-score=0.038563829787234036, device=cuda:1
INFO - 01/06/23 12:36:07 - 0:00:08 - learning: av-score=0.048537234042553175, device=cuda:0
INFO - 01/06/23 12:36:07 - 0:00:08 - learning: av-score=0.06382978723404255, device=cuda:1
INFO - 01/06/23 12:36:08 - 0:00:08 - learning: av-score=0.36768617021276595, device=cuda:0
INFO - 01/06/23 12:36:08 - 0:00:08 - learning: av-score=0.2200797872340425, device=cuda:1
INFO - 01/06/23 12:36:08 - 0:00:09 - learning: av-score=0.8138297872340422, devic

Av time per iteration per trainer: 0.69 seconds


In [10]:
torch.cuda.get_device_name(next(trainers[0].modules['transformer'].te1.parameters()).device.index)

'NVIDIA GeForce RTX 3080 Ti'

In [9]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

#rnds = [gen_rnd_chars(xy_data_size_const) for _ in range(100)]

print(trainers[0].act(A))
print(trainers[0].act(B))

AssertionError: 

In [ ]:
A = 'KAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAE'
B = 'KBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBE'
X = 'KAAADAAAAADAAAAAMAAAAAAAAAAASAAAALAAAAAFNAAZE'

losses = []
for it in range(2000):
    loss = trainer.learn(join_sai(A), join_sai(A))
    if loss[0]:
        l = loss[1].item()
        losses += [l]

plt.plot(losses)
plt.title('los s vs iteration')
plt.xlabel('iteration')
plt.ylabel('loss')
plt.show()


In [ ]:
print(act(A))

In [ ]:
print(act(B))

In [6]:
import torch.multiprocessing as mp

In [17]:
mp.set_start_method('spawn', force=True)

ValueError: cannot find context for 'fork'

In [13]:
def m():
    while True:
        print(1)

In [16]:
p = mp.Process(target=m, args=())

p.start()